In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
def preprocessing(img):
    # Resize the image
    img = cv2.resize(img, (640, 480))  # Set the desired width and height

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply a bilateral filter to smooth the image while preserving edges
    blur = cv2.bilateralFilter(gray, 9, 75, 75)

    # Apply adaptive thresholding to segment the image into foreground and background
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 15, 8)

    # Find contours in the thresholded image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    areas = [cv2.contourArea(c) for c in contours]
    max_index = np.argmax(areas)
    cnt = contours[max_index]

    # Create a mask for the foreground
    mask = np.zeros_like(thresh)
    cv2.drawContours(mask, [cnt], 0, 255, -1)

    # Apply the mask to the original image
    masked = cv2.bitwise_and(img, img, mask=mask)

    # Crop the image
    x, y, w, h = cv2.boundingRect(cnt)
    cropped = masked[y:y+h, x:x+w]

    return cropped




In [3]:
def edgeDetection(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Compute the gradients in the x and y directions using the Sobel operator
    dx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    dy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    # Compute the magnitude of the gradient and threshold the resulting image
    mag = np.sqrt(dx**2 + dy**2)
    mag_thresh = np.max(mag) * 0.1  # Set the threshold as a fraction of the maximum gradient magnitude
    edges = np.zeros_like(mag)
    edges[mag > mag_thresh] = 255

    return edges

In [4]:
def HoG(image):
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)
    return fd, hog_image

In [5]:
dataset_dir = 'dataset'
class_names = ['real', 'fake']

data = []
labels = []
n = 6000
for class_name in class_names:
    class_dir = os.path.join(dataset_dir, class_name)

    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = cv2.imread(image_path)

        preprocessed_img = preprocessing(image)
        edges_image = edgeDetection(preprocessed_img)
        features, hog_img = HoG(edges_image)

        # Check if features has the correct shape
        if features.shape != (n,):
            # Reshape or pad features to correct shape
            features = np.resize(features, n)
        data.append(features)
        labels.append(class_names.index(class_name))

In [6]:
data = np.array(data)
labels = np.array(labels)

X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

clf = SVC(kernel='linear', C=1.0, random_state=42)
clf.fit(X_train, Y_train)

Y_pred = clf.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

print(f'Accuracy: {accuracy: .2f}, Precision: {precision: .2f}, Recall: {recall: .2f}, F1-score: {f1: .2f}')

Accuracy:  1.00, Precision:  1.00, Recall:  1.00, F1-score:  1.00


In [7]:
path = 'sample.png'

test_image = cv2.imread(path)

test_preprocessed = preprocessing(test_image)

test_edges = edgeDetection(test_preprocessed)
test_features, hm = HoG(test_edges)

if test_features.shape != (n,):
    # Reshape or pad features to correct shape
    test_features = np.resize(test_features, n)

prediction = clf.predict([test_features])[0]

if prediction == 0:
    print('real')

else:
    print('fake')

fake


In [8]:
import joblib

model = clf
joblib.dump(model, 'currencyDetector.pkl')

['currencyDetector.pkl']